<a href="https://colab.research.google.com/github/BrunoStrufaldi/Automa-o-GoogleColab/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import re

# ==============================================================================
# Função para remover emojis de qualquer texto
# ==============================================================================
def remover_emojis(texto):
    """Remove emojis de uma string de texto."""
    if not texto:
        return ""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', texto).strip()
# ==============================================================================

# NOMES DOS ARQUIVOS
NOME_DO_ARQUIVO_ENTRADA = '/projeto pyton.xlsx'
NOME_DO_ARQUIVO_SAIDA = 'descricoes_finais.xlsx'
# ==============================================================================

try:
    df = pd.read_excel(NOME_DO_ARQUIVO_ENTRADA)
    print(f"--- Arquivo '{NOME_DO_ARQUIVO_ENTRADA}' lido com sucesso! ---")

    lista_de_descricoes = []

    def formatar_lista_com_emoji(lista_de_itens, emoji="📌"):
        itens_validos = [item.strip() for item in lista_de_itens if item.strip()]
        return "\n".join([f"  {emoji} {item}" for item in itens_validos])

    for index, row in df.iterrows():
        texto_bruto_da_planilha = str(row.get('descrição', '')).strip()
        descricao_bruta = remover_emojis(texto_bruto_da_planilha)

        if not descricao_bruta:
            continue

        linhas = descricao_bruta.split('\n')
        primeira_linha = linhas[0].strip()
        titulo_anuncio = primeira_linha.upper()
        linhas_para_processar = linhas[1:]

        compatibilidade_items, conteudo_items, material_items = [], [], []
        secao_atual = None

        PALAVRAS_DE_PARADA = ["validade", "benefícios", "atenção", "dúvidas", "garanta já", "prazo de validade", "indeterminado", "antes da compra"]

        for linha in linhas_para_processar:
            linha_lower = linha.lower().strip()

            if "compatibilidade" in linha_lower:
                secao_atual = 'compatibilidade'
                continue
            elif "conteúdo do kit" in linha_lower:
                secao_atual = 'conteudo'
                continue
            elif "material" in linha_lower:
                secao_atual = 'materiais'
                continue

            # Aplica as regras de cada seção
            if secao_atual == 'compatibilidade':
                if linha.strip() and not compatibilidade_items:
                    compatibilidade_items.append(linha.strip())
                secao_atual = None

            elif secao_atual == 'conteudo':
                # AJUSTE FINAL: Verifica se a linha contém uma palavra de parada ANTES de adicionar
                if any(palavra in linha_lower for palavra in PALAVRAS_DE_PARADA):
                    secao_atual = None
                    continue
                if linha.strip():
                    conteudo_items.append(linha.strip().lstrip('✔').strip())

            elif secao_atual == 'materiais':
                if ":" in linha and not any(palavra in linha_lower for palavra in PALAVRAS_DE_PARADA):
                    material_items.append(linha.strip())
                else:
                    secao_atual = None

        compatibilidade_formatado = ""
        if compatibilidade_items:
            itens_compat_str = formatar_lista_com_emoji(compatibilidade_items, emoji="📌")
            compatibilidade_formatado = f"\n\n🔗 **COMPATIBILIDADE:**\n{itens_compat_str}"

        conteudo_formatado = formatar_lista_com_emoji(conteudo_items)
        conteudo_kit_raw = " ".join(conteudo_items)
        conteudo_lower_completo = (titulo_anuncio + conteudo_kit_raw).lower()

        if not material_items:
            inferred_materials = []
            if "queimador" in conteudo_lower_completo: inferred_materials.append("Queimadores: Alumínio")
            if "espalhador" in conteudo_lower_completo: inferred_materials.append("Espalhadores: Aço Esmaltado")
            if "grelha" in conteudo_lower_completo: inferred_materials.append("Grelhas: Arame De Aço Esmaltado")
            material_formatado = formatar_lista_com_emoji(inferred_materials)
        else:
            material_formatado = formatar_lista_com_emoji(material_items)

        beneficios = ["Chamas mais estáveis e aquecimento uniforme", "Alta durabilidade", "Fácil instalação", "Reposição perfeita"]
        if "SÓ GRELHA" in conteudo_kit_raw.upper() or ("GRELHA" in conteudo_kit_raw.upper() and len(conteudo_items) <= 2):
            beneficios.remove("Chamas mais estáveis e aquecimento uniforme")
        beneficios_formatado = "".join([f"📌 {b}\n" for b in beneficios])

        TODOS_COMPONENTES = {"espalhadores", "queimadores", "grelhas"}
        componentes_presentes = set()
        if "espalhador" in conteudo_lower_completo: componentes_presentes.add("espalhadores")
        if "queimador" in conteudo_lower_completo: componentes_presentes.add("queimadores")
        if "grelha" in conteudo_lower_completo: componentes_presentes.add("grelhas")

        componentes_ausentes = sorted(list(TODOS_COMPONENTES - componentes_presentes))

        obs_final = ""
        if len(componentes_ausentes) == 2:
            obs_final = f"Obs: este anúncio não inclui {componentes_ausentes[0]} e {componentes_ausentes[1]}."
        elif len(componentes_ausentes) == 1:
            obs_final = f"Obs: este anúncio não inclui {componentes_ausentes[0]}."
        elif len(componentes_presentes) > 0 and len(componentes_ausentes) == 0:
             obs_final = "Obs: anúncio referente a um kit completo com queimadores, espalhadores e grelhas."
        else:
            itens_str = " e ".join(sorted(list(componentes_presentes)))
            artigo = "aos"
            if "grelhas" in itens_str and len(componentes_presentes) == 1:
                artigo = "às"
            obs_final = f"Obs: este anúncio se refere apenas {artigo} {itens_str}."

        descricao_final = f"""🔥 **{titulo_anuncio}** 🔥

🔹 Renove seu fogão com peças de **alta qualidade, encaixe perfeito e longa durabilidade!**
✅ Resistência e acabamento premium garantidos.{compatibilidade_formatado}

📦 **CONTEÚDO DO KIT:**
{conteudo_formatado}

⚙️ **MATERIAL:**
{material_formatado}

🕒 **PRAZO DE VALIDADE:**
📌 Indeterminado

💪 **BENEFÍCIOS:**
{beneficios_formatado}
⚠️ **ATENÇÃO:**
📌 Antes da compra, compare as fotos, medidas e quantidades de peças do anúncio com as peças do seu fogão para garantir o encaixe correto.
📌 {obs_final}

📩 **DÚVIDAS?**
📌 Estamos à disposição para ajudar. Entre em contato pelo campo de mensagens que teremos prazer em responder.

🔥 Garanta já o seu kit e deixe seu fogão como novo!
"""

        lista_de_descricoes.append(descricao_final)

    print("\n--- Criando a planilha Excel com os resultados... ---")
    df_saida = pd.DataFrame(lista_de_descricoes, columns=['Descrições Geradas'])
    df_saida.to_excel(NOME_DO_ARQUIVO_SAIDA, index=False)

    print(f"\n--- FIM: Todas as descrições foram geradas e salvas em '{NOME_DO_ARQUIVO_SAIDA}'. ---")
    print("--- Você pode baixar a planilha na aba 'Arquivos' à esquerda. ---")

except FileNotFoundError:
    print(f"\n### ERRO: Arquivo '{NOME_DO_ARQUIVO_ENTRADA}' não encontrado. Verifique o nome e se fez o upload. ###")
except Exception as e:
    print(f"\n### ERRO INESPERADO: {e} ###")

--- Arquivo '/projeto pyton.xlsx' lido com sucesso! ---

--- Criando a planilha Excel com os resultados... ---

--- FIM: Todas as descrições foram geradas e salvas em 'descricoes_finais.xlsx'. ---
--- Você pode baixar a planilha na aba 'Arquivos' à esquerda. ---
